In [ ]:
#pip install ipykernel

In [ ]:
#python -m ipykernel install --user --name=fakeNewsDetectionEnv --display-name "Python (fakeNewsDetectionEnv)"

In [1]:
# pip install transformers

# pip install torch torchvision torchaudio
# Install PyTorch XLA for TPUs (usually needed on Kaggle or custom setups)
!pip install torch-xla -f https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

Looking in links: https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import gc

gc.collect()

0

In [3]:
import torch

torch.cuda.empty_cache()  # Use this after every epoch or large memory allocation

In [5]:
# from google.colab import drive

# drive.mount('/content/drive')

In [27]:
import pandas as pd

In [28]:
gossip_fake = pd.read_csv('/kaggle/input/fakenewsdataset/FakeNewsNet/dataset/gossipcop_fake.csv')

gossip_real = pd.read_csv('/kaggle/input/fakenewsdataset/FakeNewsNet/dataset/gossipcop_real.csv')

politifact_fake = pd.read_csv('/kaggle/input/fakenewsdataset/FakeNewsNet/dataset/politifact_fake.csv')

politifact_real = pd.read_csv('/kaggle/input/fakenewsdataset/FakeNewsNet/dataset/politifact_real.csv')

In [29]:
gossip_fake['label'] = 1

gossip_real['label'] = 0

politifact_fake['label'] = 1

politifact_real['label'] = 0

In [30]:
# Combine fake and real news for both domains

gossip = pd.concat([gossip_fake[['title', 'label']], gossip_real[['title', 'label']]])

politifact = pd.concat([politifact_fake[['title', 'label']], politifact_real[['title', 'label']]])

In [31]:
gossip

,title,label
0,Did Miley Cyrus and Liam Hemsworth secretly ge...,1
1,Paris Jackson & Cara Delevingne Enjoy Night Ou...,1
2,Celebrities Join Tax March in Protest of Donal...,1
3,Cindy Crawford's daughter Kaia Gerber wears a ...,1
4,Full List of 2018 Oscar Nominations – Variety,1
...,...,...
16812,2017 Hollywood Film Awards: The Complete List ...,0
16813,Jada Pinkett Smith explains why son Jaden move...,0
16814,Tinsley Mortimer Reacts to Luann de Lesseps' R...,0
16815,Prince Harry Carries on Princess Diana’s Legac...,0


In [32]:
news_df = pd.concat([gossip, politifact], axis=0).reset_index(drop=True)

In [33]:
news_df.head(5)

,title,label
0,Did Miley Cyrus and Liam Hemsworth secretly ge...,1
1,Paris Jackson & Cara Delevingne Enjoy Night Ou...,1
2,Celebrities Join Tax March in Protest of Donal...,1
3,Cindy Crawford's daughter Kaia Gerber wears a ...,1
4,Full List of 2018 Oscar Nominations – Variety,1


In [35]:
import re



def clean_text(text):

    # Remove special characters and digits, and convert text to lowercase

    text = re.sub(r'\W', ' ', text)

    text = re.sub(r'\d', ' ', text)

    text = text.lower()

    return text



# Apply cleaning to all articles

news_df['title'] = news_df['title'].apply(clean_text)


In [10]:
from transformers import BertTokenizer



# Load the pre-trained BERT tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



# Tokenize the dataset

def tokenize_function(examples):

    return tokenizer(examples, padding='max_length', truncation=True, max_length=512)



# Apply the tokenizer to the text column

news_df['input_ids'] = news_df['title'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512)['input_ids'])

news_df['attention_mask'] = news_df['title'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512)['attention_mask'])


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
W1114 05:31:52.973000 140635694196416 torch/_inductor/compile_worker/subproc_pool.py:126] SubprocPool unclean exit


In [11]:
news_df.head(5)

,title,label,input_ids,attention_mask
0,did miley cyrus and liam hemsworth secretly ge...,1,"[101, 2106, 3542, 2100, 16123, 1998, 8230, 196...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
1,paris jackson cara delevingne enjoy night ou...,1,"[101, 3000, 4027, 14418, 3972, 23559, 2638, 59...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,celebrities join tax march in protest of donal...,1,"[101, 12330, 3693, 4171, 2233, 1999, 6186, 199...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
3,cindy crawford s daughter kaia gerber wears a ...,1,"[101, 15837, 10554, 1055, 2684, 11928, 2050, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,full list of oscar nominations variety,1,"[101, 2440, 2862, 1997, 7436, 9930, 3528, 102,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
news_df.shape

(23196, 4)

In [13]:
from sklearn.model_selection import train_test_split



# Split the data into training and validation sets

train_texts, val_texts, train_labels, val_labels = train_test_split(news_df[['input_ids', 'attention_mask']], news_df['label'], test_size=0.1)

In [14]:
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler



# Convert tokenized data into PyTorch tensors

train_inputs = torch.tensor([x for x in train_texts['input_ids']])

train_masks = torch.tensor([x for x in train_texts['attention_mask']])

train_labels = torch.tensor(train_labels.values)



val_inputs = torch.tensor([x for x in val_texts['input_ids']])

val_masks = torch.tensor([x for x in val_texts['attention_mask']])

val_labels = torch.tensor(val_labels.values)



# Create DataLoader for training and validation sets

batch_size = 4



train_data = TensorDataset(train_inputs, train_masks, train_labels)

train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)



val_data = TensorDataset(val_inputs, val_masks, val_labels)

val_sampler = SequentialSampler(val_data)

val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


In [22]:
import torch
import time
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.cuda.amp import GradScaler  # Only use GradScaler without autocast for TPU

# Define the main training function
def train_tpu():
    # Setup device (TPU core)
    device = xm.xla_device()
    
    # Load BERT model and move it to TPU device
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
    
    # Set up optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=2e-5)
    epochs = 3
    
    # DataLoader for training and validation
    train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=4)
    val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data), batch_size=4)
    
    # Total training steps and scheduler
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    scaler = GradScaler()  # Mixed precision

    # Training loop
    for epoch in range(epochs):
        start_epoch_time = time.time()  # Record start time for the epoch
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training phase
        model.train()
        total_loss = 0
        para_loader = pl.ParallelLoader(train_dataloader, [device])
        for step, batch in enumerate(para_loader.per_device_loader(device)):
            start_batch_time = time.time()  # Record start time for the batch
            batch_input_ids, batch_input_mask, batch_labels = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass without autocast
            outputs = model(batch_input_ids, attention_mask=batch_input_mask, labels=batch_labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Backward pass with scaled gradients
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            xm.optimizer_step(optimizer, barrier=True)
            scheduler.step()

            # Log the current batch time
            batch_time = time.time() - start_batch_time
            if step % 100 == 0:  # Log every 100 steps
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{step}/{len(train_dataloader)}], Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

        # Log the epoch time
        epoch_time = time.time() - start_epoch_time
        print(f"Epoch [{epoch+1}/{epochs}] completed in {epoch_time:.2f} seconds.")

        # Validation phase
        model.eval()
        eval_loss = 0
        eval_accuracy = 0
        para_loader = pl.ParallelLoader(val_dataloader, [device])
        for batch in para_loader.per_device_loader(device):
            batch_input_ids, batch_input_mask, batch_labels = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )

            with torch.no_grad():
                outputs = model(batch_input_ids, attention_mask=batch_input_mask, labels=batch_labels)
                loss = outputs.loss
                eval_loss += loss.item()

                # Predictions for accuracy
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1)
                eval_accuracy += accuracy_score(batch_labels.cpu(), preds.cpu())

        # Average validation loss and accuracy
        avg_val_loss = eval_loss / len(val_dataloader)
        avg_val_accuracy = eval_accuracy / len(val_dataloader)
        xm.master_print(f"Validation loss: {avg_val_loss:.4f}, Validation accuracy: {avg_val_accuracy:.4f}")

    # Return the trained model
    return model

# Run training and get the trained model
trained_model = train_tpu()
trained_model.to("cpu")

# After training is finished, save the model
print("Saving the fine-tuned model...")
trained_model.save_pretrained('./fake_news_model')
tokenizer.save_pretrained('./fake_news_tokenizer')
print("Model and Tokenizer saved to './fake_news_model'.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_13/1247459758.py:29: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed precision


Epoch 1/3
Epoch [1/3], Batch [0/5219], Loss: 0.6816, Time: 0.70 seconds
Epoch [1/3], Batch [100/5219], Loss: 0.5717, Time: 0.73 seconds
Epoch [1/3], Batch [200/5219], Loss: 0.7850, Time: 0.74 seconds
Epoch [1/3], Batch [300/5219], Loss: 0.3562, Time: 0.71 seconds
Epoch [1/3], Batch [400/5219], Loss: 0.2894, Time: 0.72 seconds
Epoch [1/3], Batch [500/5219], Loss: 0.1384, Time: 0.72 seconds
Epoch [1/3], Batch [600/5219], Loss: 0.4608, Time: 0.73 seconds
Epoch [1/3], Batch [700/5219], Loss: 0.2784, Time: 0.72 seconds
Epoch [1/3], Batch [800/5219], Loss: 1.5725, Time: 0.72 seconds
Epoch [1/3], Batch [900/5219], Loss: 0.8742, Time: 0.73 seconds
Epoch [1/3], Batch [1000/5219], Loss: 0.2002, Time: 0.73 seconds
Epoch [1/3], Batch [1100/5219], Loss: 0.8140, Time: 0.72 seconds
Epoch [1/3], Batch [1200/5219], Loss: 0.7809, Time: 0.73 seconds
Epoch [1/3], Batch [1300/5219], Loss: 0.1786, Time: 0.72 seconds
Epoch [1/3], Batch [1400/5219], Loss: 0.0647, Time: 0.72 seconds
Epoch [1/3], Batch [1500/52

In [17]:
import shutil

# Zip the model and tokenizer directories
shutil.make_archive("/kaggle/working/fake_news_model", 'zip', "./fake_news_model")
shutil.make_archive("/kaggle/working/fake_news_tokenizer", 'zip', "./fake_news_tokenizer")

print("Model and tokenizer have been zipped.")

Model and tokenizer have been zipped.


In [24]:
torch.save(trained_model.state_dict(), '/kaggle/working/model_state.pth')

In [18]:
from IPython.display import FileLink
FileLink(r'fake_news_model.zip')

/kaggle/working/fake_news_model.zip

In [30]:
FileLink(r'model_state.pth')

/kaggle/working/model_state.pth

In [34]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

# Load test set
test_df = pd.read_csv('/kaggle/input/test-set/train.csv')

# Extract the top 50 rows
test_df = test_df.head(50)

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('./fake_news_tokenizer')
model = BertForSequenceClassification.from_pretrained('./fake_news_model')
model.eval()

# Preprocess the test data
inputs = tokenizer(list(test_df['text']), padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Calculate metrics
accuracy = accuracy_score(test_df['label'], predictions)
report = classification_report(test_df['label'], predictions, target_names=['Class 0', 'Class 1'])

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)


Accuracy: 0.5200
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.86      0.21      0.33        29
     Class 1       0.47      0.95      0.62        21

    accuracy                           0.52        50
   macro avg       0.66      0.58      0.48        50
weighted avg       0.69      0.52      0.46        50



In [35]:
# processing

In [36]:
# Assuming your dataset is in a DataFrame named 'train_data' and the label column is named 'label'
label_counts = news_df['label'].value_counts()
print("Class distribution in training data:")
print(label_counts)

Class distribution in training data:
label
0    17441
1     5755
Name: count, dtype: int64


In [16]:
import torch
import time
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.cuda.amp import GradScaler  # Only use GradScaler without autocast for TPU

# Define the main training function
def train_tpu():
    # Setup device (TPU core)
    device = xm.xla_device()

    # Calculate weights for each class
    class_weights = torch.tensor([1.0, 17441 / 5755], device=device)  # Modify this ratio based on your counts
    
    # Load BERT model and move it to TPU device
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
    
    # Set up optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=2e-5)
    epochs = 3
    
    # DataLoader for training and validation
    train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=4)
    val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data), batch_size=4)
    
    # Total training steps and scheduler
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    scaler = GradScaler()  # Mixed precision

    # Training loop
    for epoch in range(epochs):
        start_epoch_time = time.time()  # Record start time for the epoch
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training phase
        model.train()
        total_loss = 0
        para_loader = pl.ParallelLoader(train_dataloader, [device])
        for step, batch in enumerate(para_loader.per_device_loader(device)):
            start_batch_time = time.time()  # Record start time for the batch
            batch_input_ids, batch_input_mask, batch_labels = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_input_ids, attention_mask=batch_input_mask, labels=batch_labels)
            loss = outputs.loss

            # Apply class weights to the loss
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
            loss = loss_fct(outputs.logits, batch_labels)  # Use logits and labels directly for the loss calculation

            total_loss += loss.item()

            # Backward pass with scaled gradients
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            xm.optimizer_step(optimizer, barrier=True)
            scheduler.step()

            # Log the current batch time
            batch_time = time.time() - start_batch_time
            if step % 100 == 0:  # Log every 100 steps
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{step}/{len(train_dataloader)}], Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

        # Log the epoch time
        epoch_time = time.time() - start_epoch_time
        print(f"Epoch [{epoch+1}/{epochs}] completed in {epoch_time:.2f} seconds.")

        # Validation phase
        model.eval()
        eval_loss = 0
        eval_accuracy = 0
        para_loader = pl.ParallelLoader(val_dataloader, [device])
        for batch in para_loader.per_device_loader(device):
            batch_input_ids, batch_input_mask, batch_labels = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )

            with torch.no_grad():
                outputs = model(batch_input_ids, attention_mask=batch_input_mask, labels=batch_labels)
                loss = outputs.loss

                # Apply class weights to the loss
                loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
                loss = loss_fct(outputs.logits, batch_labels)

                eval_loss += loss.item()

                # Predictions for accuracy
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1)
                eval_accuracy += accuracy_score(batch_labels.cpu(), preds.cpu())

        # Average validation loss and accuracy
        avg_val_loss = eval_loss / len(val_dataloader)
        avg_val_accuracy = eval_accuracy / len(val_dataloader)
        xm.master_print(f"Validation loss: {avg_val_loss:.4f}, Validation accuracy: {avg_val_accuracy:.4f}")

    # Return the trained model
    return model

# Run training and get the trained model
trained_model = train_tpu()
trained_model.to("cpu")

# After training is finished, save the model
print("Saving the fine-tuned model...")
trained_model.save_pretrained('./fake_news_model')
tokenizer.save_pretrained('./fake_news_tokenizer')
print("Model and Tokenizer saved to './fake_news_model'.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_13/2767510910.py:32: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed precision


Epoch 1/3
Epoch [1/3], Batch [0/5219], Loss: 0.8029, Time: 39.38 seconds
Epoch [1/3], Batch [100/5219], Loss: 0.6714, Time: 0.72 seconds
Epoch [1/3], Batch [200/5219], Loss: 0.6456, Time: 0.75 seconds
Epoch [1/3], Batch [300/5219], Loss: 0.8562, Time: 0.75 seconds
Epoch [1/3], Batch [400/5219], Loss: 0.8011, Time: 0.70 seconds
Epoch [1/3], Batch [500/5219], Loss: 0.5158, Time: 0.72 seconds
Epoch [1/3], Batch [600/5219], Loss: 0.9125, Time: 0.71 seconds
Epoch [1/3], Batch [700/5219], Loss: 0.2648, Time: 0.71 seconds
Epoch [1/3], Batch [800/5219], Loss: 0.7612, Time: 0.74 seconds
Epoch [1/3], Batch [900/5219], Loss: 1.0252, Time: 0.71 seconds
Epoch [1/3], Batch [1000/5219], Loss: 0.3916, Time: 0.72 seconds
Epoch [1/3], Batch [1100/5219], Loss: 0.2998, Time: 0.70 seconds
Epoch [1/3], Batch [1200/5219], Loss: 0.3034, Time: 0.70 seconds
Epoch [1/3], Batch [1300/5219], Loss: 0.2480, Time: 0.74 seconds
Epoch [1/3], Batch [1400/5219], Loss: 0.1953, Time: 0.71 seconds
Epoch [1/3], Batch [1500/5

In [19]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

# Load test set
test_df = pd.read_csv('/kaggle/input/test-set/train.csv')

# Extract the top 50 rows
test_df = test_df.head(50)

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('./fake_news_tokenizer')
model = BertForSequenceClassification.from_pretrained('./fake_news_model')
model.eval()

# Preprocess the test data
inputs = tokenizer(list(test_df['text']), padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Calculate metrics
accuracy = accuracy_score(test_df['label'], predictions)
report = classification_report(test_df['label'], predictions, target_names=['Class 0', 'Class 1'])

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.4400
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.60      0.10      0.18        29
     Class 1       0.42      0.90      0.58        21

    accuracy                           0.44        50
   macro avg       0.51      0.50      0.38        50
weighted avg       0.53      0.44      0.34        50



In [ ]:
# extend the dataset

In [2]:
import pandas as pd
fake = pd.read_csv('/kaggle/input/new-fake-news/Fake.csv')
real = pd.read_csv('/kaggle/input/new-fake-news/True.csv')

In [3]:
fake_df = pd.DataFrame(fake['title'] + fake['text'], columns=['title'])
real_df = pd.DataFrame(real['title'] + real['text'], columns=['title'])

In [4]:
fake_df

,title
0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...
...,...
23476,McPain: John McCain Furious That Iran Treated ...
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479,How to Blow $700 Million: Al Jazeera America F...


In [5]:
fake_df['label']=1
real_df['label']=0

In [6]:
news_df_2 = pd.concat([fake_df, real_df], axis=0).reset_index(drop=True)

In [7]:
news_df_2.tail()

,title,label
44893,'Fully committed' NATO backs new U.S. approach...,0
44894,LexisNexis withdrew two products from Chinese ...,0
44895,Minsk cultural hub becomes haven from authorit...,0
44896,Vatican upbeat on possibility of Pope Francis ...,0
44897,Indonesia to buy $1.14 billion worth of Russia...,0


In [10]:
import re



def clean_text(text):

    # Remove special characters and digits, and convert text to lowercase

    text = re.sub(r'\W', ' ', text)

    text = re.sub(r'\d', ' ', text)

    text = text.lower()

    return text

news_df_2['title'] = news_df_2['title'].apply(clean_text)

In [11]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



# Tokenize the dataset

def tokenize_function(examples):

    return tokenizer(examples, padding='max_length', truncation=True, max_length=512)

news_df_2['input_ids'] = news_df_2['title'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512)['input_ids'])

news_df_2['attention_mask'] = news_df_2['title'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512)['attention_mask'])


In [12]:
news_df_2.shape

(44898, 4)

In [13]:
# Assuming your dataset is in a DataFrame named 'train_data' and the label column is named 'label'
label_counts = news_df_2['label'].value_counts()
print("Class distribution in training data:")
print(label_counts)

Class distribution in training data:
label
1    23481
0    21417
Name: count, dtype: int64


In [14]:
from sklearn.model_selection import train_test_split



# Split the data into training and validation sets

train_texts, val_texts, train_labels, val_labels = train_test_split(news_df_2[['input_ids', 'attention_mask']], news_df_2['label'], test_size=0.1)

In [15]:
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler



# Convert tokenized data into PyTorch tensors

train_inputs = torch.tensor([x for x in train_texts['input_ids']])

train_masks = torch.tensor([x for x in train_texts['attention_mask']])

train_labels = torch.tensor(train_labels.values)



val_inputs = torch.tensor([x for x in val_texts['input_ids']])

val_masks = torch.tensor([x for x in val_texts['attention_mask']])

val_labels = torch.tensor(val_labels.values)



# Create DataLoader for training and validation sets

batch_size = 4



train_data = TensorDataset(train_inputs, train_masks, train_labels)

train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)



val_data = TensorDataset(val_inputs, val_masks, val_labels)

val_sampler = SequentialSampler(val_data)

val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


In [16]:
import torch
import time
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.cuda.amp import GradScaler  # Only use GradScaler without autocast for TPU

# Define the main training function
def train_tpu():
    # Setup device (TPU core)
    device = xm.xla_device()

    # Load BERT model and move it to TPU device
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', num_labels=2
    ).to(device)

    # Hyperparameters
    learning_rate = 2e-5
    batch_size = 4
    accumulation_steps = 4  # To simulate larger batch size
    epochs = 3

    # Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)
    val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data), batch_size=batch_size)

    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps
    )
    scaler = GradScaler()

    # Training loop
    for epoch in range(epochs):
        start_epoch_time = time.time()
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training phase
        model.train()
        total_loss = 0
        para_loader = pl.ParallelLoader(train_dataloader, [device])

        for step, batch in enumerate(para_loader.per_device_loader(device)):
            batch_input_ids, batch_input_mask, batch_labels = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_input_ids, attention_mask=batch_input_mask, labels=batch_labels)
            loss = outputs.loss / accumulation_steps  # Normalize loss for gradient accumulation
            total_loss += loss.item() * accumulation_steps

            # Backward pass
            scaler.scale(loss).backward()

            if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                # Optimizer and Scheduler steps
                scaler.step(optimizer)
                scaler.update()
                xm.optimizer_step(optimizer, barrier=True)
                scheduler.step()
                optimizer.zero_grad()

            # Log progress
            if step % 100 == 0:
                avg_loss = total_loss / ((step + 1) * batch_size)
                print(f"Step [{step}/{len(train_dataloader)}], Avg Loss: {avg_loss:.4f}")

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch [{epoch + 1}/{epochs}] Training Loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        eval_loss = 0
        eval_accuracy = 0
        para_loader = pl.ParallelLoader(val_dataloader, [device])

        for batch in para_loader.per_device_loader(device):
            batch_input_ids, batch_input_mask, batch_labels = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )

            with torch.no_grad():
                outputs = model(batch_input_ids, attention_mask=batch_input_mask, labels=batch_labels)
                loss = outputs.loss
                eval_loss += loss.item()

                # Predictions for accuracy
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1)
                eval_accuracy += accuracy_score(batch_labels.cpu(), preds.cpu())

        avg_val_loss = eval_loss / len(val_dataloader)
        avg_val_accuracy = eval_accuracy / len(val_dataloader)
        xm.master_print(f"Validation loss: {avg_val_loss:.4f}, Validation accuracy: {avg_val_accuracy:.4f}")

    # Return the trained model
    return model

# Run training and get the trained model
trained_model = train_tpu()
trained_model.to("cpu")

# Save the trained model and tokenizer
print("Saving the fine-tuned model...")
trained_model.save_pretrained('./fake_news_model')
tokenizer.save_pretrained('./fake_news_tokenizer')
print("Model and Tokenizer saved to './fake_news_model'.")


E0000 00:00:1731678281.545088      13 common_lib.cc:818] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:483
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_13/3885354821.py:35: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Pleas

Epoch 1/3
Step [0/10102], Avg Loss: 0.1811
Step [100/10102], Avg Loss: 0.1756
Step [200/10102], Avg Loss: 0.1758
Step [300/10102], Avg Loss: 0.1755
Step [400/10102], Avg Loss: 0.1757
Step [500/10102], Avg Loss: 0.1758
Step [600/10102], Avg Loss: 0.1754
Step [700/10102], Avg Loss: 0.1751
Step [800/10102], Avg Loss: 0.1751
Step [900/10102], Avg Loss: 0.1752
Step [1000/10102], Avg Loss: 0.1752
Step [1100/10102], Avg Loss: 0.1751
Step [1200/10102], Avg Loss: 0.1750
Step [1300/10102], Avg Loss: 0.1750
Step [1400/10102], Avg Loss: 0.1750
Step [1500/10102], Avg Loss: 0.1749
Step [1600/10102], Avg Loss: 0.1748
Step [1700/10102], Avg Loss: 0.1748
Step [1800/10102], Avg Loss: 0.1749
Step [1900/10102], Avg Loss: 0.1747
Step [2000/10102], Avg Loss: 0.1746
Step [2100/10102], Avg Loss: 0.1746
Step [2200/10102], Avg Loss: 0.1747
Step [2300/10102], Avg Loss: 0.1746
Step [2400/10102], Avg Loss: 0.1746
Step [2500/10102], Avg Loss: 0.1746
Step [2600/10102], Avg Loss: 0.1745
Step [2700/10102], Avg Loss: 0

In [17]:
import shutil

# Zip the model and tokenizer directories
shutil.make_archive("/kaggle/working/fake_news_model", 'zip', "./fake_news_model")
shutil.make_archive("/kaggle/working/fake_news_tokenizer", 'zip', "./fake_news_tokenizer")

print("Model and tokenizer have been zipped.")

Model and tokenizer have been zipped.


In [23]:
from IPython.display import FileLink
FileLink(r'fake_news_model.zip')

/kaggle/working/fake_news_model.zip

In [19]:
test = pd.read_csv('/kaggle/input/test-set-3/news.csv')
test_data = test['title'] + test['text']
test_data = pd.DataFrame(test_data, columns=['text'])
test_label = pd.DataFrame(test['label'].replace({'FAKE': 1, 'REAL': 0}), columns=['label'])
test = pd.concat([test_data, test_label], axis=1).reset_index(drop=True)

/tmp/ipykernel_13/1945978785.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = pd.DataFrame(test['label'].replace({'FAKE': 1, 'REAL': 0}), columns=['label'])


In [44]:
news_df.head(10)

,title,label
0,did miley cyrus and liam hemsworth secretly ge...,1
1,paris jackson cara delevingne enjoy night ou...,1
2,celebrities join tax march in protest of donal...,1
3,cindy crawford s daughter kaia gerber wears a ...,1
4,full list of oscar nominations variety,1
5,here s what really happened when jfk jr met p...,1
6,biggest celebrity scandals of,1
7,caitlyn jenner addresses rumored romance with ...,1
8,taylor swift reportedly reacts to tom hiddlest...,1
9,for the love of god why can t anyone write ka...,1


In [49]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

# Load test set
test_df = test.head(10)

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('/kaggle/working/fake_news_tokenizer')
model = BertForSequenceClassification.from_pretrained('/kaggle/working/fake_news_model')
model.eval()

# Preprocess the test data
inputs = tokenizer(list(test_df['text']), padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Calculate metrics
accuracy = accuracy_score(test_df['label'], predictions)
report = classification_report(test_df['label'], predictions, target_names=['Class 0', 'Class 1'])

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.9000
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.83      1.00      0.91         5
     Class 1       1.00      0.80      0.89         5

    accuracy                           0.90        10
   macro avg       0.92      0.90      0.90        10
weighted avg       0.92      0.90      0.90        10

